### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [2]:
print(movies.head())
print(reviews.head())

   movie_id                                              movie  \
0         8      Edison Kinetoscopic Record of a Sneeze (1894)   
1        10                La sortie des usines Lumière (1895)   
2        12                      The Arrival of a Train (1896)   
3        25  The Oxford and Cambridge University Boat Race ...   
4        91                         Le manoir du diable (1896)   

               genre  date  1800's  1900's  2000's  History  News  Horror  \
0  Documentary|Short  1894       1       0       0        0     0       0   
1  Documentary|Short  1895       1       0       0        0     0       0   
2  Documentary|Short  1896       1       0       0        0     0       0   
3                NaN  1895       1       0       0        0     0       0   
4       Short|Horror  1896       1       0       0        0     0       1   

     ...     Fantasy  Romance  Game-Show  Action  Documentary  Animation  \
0    ...           0        0          0       0            1   

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [3]:
def popular_recommendations(user_id, n_top):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    # Ratings of each movie
    movie_ratings = reviews.groupby('movie_id')['rating']
    movie_ratings_mean = movie_ratings.mean()
    movie_ratings_count = movie_ratings.count()
    movie_ratings_recent = reviews.groupby('movie_id')['date'].agg('max') # last
    
    # merge three criteria results
    movie_ranking = pd.DataFrame({'avg_rating': movie_ratings_mean, 
                                  'num_ratings': movie_ratings_count, 
                                  'last_rating': movie_ratings_recent}).reset_index()
    
    # merge with the movies dataframe
    movie_recommendation = movies.merge(movie_ranking, on = 'movie_id', how = 'inner')
    
    # sort movies' information based on the three criteria 
    movie_recommendation = movie_recommendation.sort_values(['avg_rating', 'num_ratings', 'last_rating'], 
                                                            ascending = False)
    
    # drop movies if the number of ratings are less than 5
    movie_recommendation = movie_recommendation[movie_recommendation.num_ratings > 4]
    
    top_movies = movie_recommendation.movie[:n_top].tolist()
    
    return top_movies # a list of the n_top movies as recommended

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [4]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations('1', 20) # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations('53968', 5)# Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations('70000', 100)# Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations('43', 35)# Your solution list here



In [6]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```

In [8]:
def create_ranked_df(movies, reviews):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    ranked_movies - a dataframe of the recommended movies
    '''
    # Do stuff
    # Ratings of each movie
    movie_ratings = reviews.groupby('movie_id')['rating']
    movie_ratings_mean = movie_ratings.mean()
    movie_ratings_count = movie_ratings.count()
    movie_ratings_recent = reviews.groupby('movie_id')['date'].agg('max') # last
    
    # merge three criteria results
    movie_ranking = pd.DataFrame({'avg_rating': movie_ratings_mean, 
                                  'num_ratings': movie_ratings_count, 
                                  'last_rating': movie_ratings_recent}).reset_index()
    
    # merge with the movies dataframe
    ranked_movies = movies.merge(movie_ranking, on = 'movie_id', how = 'inner')
    
    # sort movies' information based on the three criteria 
    ranked_movies = ranked_movies.sort_values(['avg_rating', 'num_ratings', 'last_rating'], 
                                              ascending = False)
    
    # drop movies if the number of ratings are less than 5
    ranked_movies = ranked_movies[ranked_movies.num_ratings > 4]
        
    return ranked_movies 


def popular_recs_filtered(user_id, n_top, ranked_movies, years=None, genres=None):
    '''    
    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies
    
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Filter movies based on year and genre
    if years is not None:
        ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]

    if genres is not None:
        num_genre_match = ranked_movies[genres].sum(axis=1)
        ranked_movies = ranked_movies.loc[num_genre_match > 0, :]
            
            
    # create top movies list 
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies

In [10]:
ranked_movies = create_ranked_df(movies, reviews)

recs_20_for_1 = popular_recs_filtered('1', 20, ranked_movies, 
                                      years=['2015', '2016', '2017', '2018'], 
                                      genres=['History'])

In [11]:
assert t.popular_recs_filtered('1', 20, 
                               ranked_movies, years=['2015', '2016', '2017', '2018'], 
                               genres=['History']) == recs_20_for_1,  "The first check failed..."
